In [22]:
import numpy as np
import pandas as pd
from datetime import timedelta
from sklearn.feature_selection import mutual_info_regression
import plotly.express as px

In [23]:
residuals = pd.read_parquet("/home/thomas/repos/simplify_deployment/data/residuals/residuals_s1_50_gen_parallel.parquet")
residuals

,residuals_s1_vars_50_gen
2022-12-01 21:44:00+00:00,-39.519329
2022-12-01 21:59:00+00:00,96.451439
2022-12-01 22:14:00+00:00,-203.770171
2022-12-01 22:29:00+00:00,-119.853498
2022-12-01 22:44:00+00:00,0.045247
...,...
2023-10-31 21:29:00+00:00,167.930012
2023-10-31 21:44:00+00:00,30.995578
2023-10-31 21:59:00+00:00,-7.644297
2023-10-31 22:14:00+00:00,20.902485


In [24]:
start = min(residuals.index)
stop = max(residuals.index)

In [25]:
assets = pd.read_parquet("/home/thomas/repos/simplify_deployment/data/lots_of_vars/asset_data.parquet")
assets = assets.sort_index(ascending = True).loc[start:stop,:].fillna(0)
assets

,Wind 2 Production,Wind 2 Schedule,Wind 2 mFRR,Wind 2 Deviation,Wind 11 Production,Wind 11 Schedule,Wind 11 mFRR,Wind 11 Deviation,Wind 9 Production,Wind 9 Schedule,...,Hydro 1 - Unit 2 Deviation reactivity,Hydro 1 - Unit 3 Deviation reactivity,Hydro 1 - Unit 4 Deviation reactivity,Hydro 1 - Unit 5 Deviation reactivity,Hydro 1 - Unit 6 Deviation reactivity,Hydro 2 - Unit 1 Deviation reactivity,Hydro 2 - Unit 2 Deviation reactivity,Hydro 2 - Unit 3 Deviation reactivity,Hydro 2 - Unit 4 Deviation reactivity,siCumulative reactivity
Step date time,,,,,,,,,,,,,,,,,,,,,
2022-12-01 21:44:00+00:00,-7.015717,8.6,0.0,-15.615717,13.002380,11.4,0.0,1.602380,9.567457,23.8,...,310428.515857,310428.515857,310428.515857,310428.515857,310428.515857,310428.515857,310428.515857,310428.515857,310428.515857,9447.636915
2022-12-01 21:45:00+00:00,-7.015717,7.5,0.0,-14.515717,12.238272,10.0,0.0,2.238272,9.863822,21.6,...,257858.918055,257858.918055,257858.918055,257858.918055,257858.918055,257858.918055,257858.918055,257858.918055,257858.918055,8794.648295
2022-12-01 21:46:00+00:00,-7.015717,7.5,0.0,-14.515717,11.659821,10.0,0.0,1.659821,9.860260,21.6,...,197225.888436,197225.888436,197225.888436,197225.888436,197225.888436,197225.888436,197225.888436,197225.888436,197225.888436,5623.431914
2022-12-01 21:47:00+00:00,-7.015717,7.5,0.0,-14.515717,12.216827,10.0,0.0,2.216827,9.146121,21.6,...,134440.772917,134440.772917,134440.772917,134440.772917,134440.772917,134440.772917,134440.772917,134440.772917,134440.772917,5467.511692
2022-12-01 21:48:00+00:00,-7.015717,7.5,0.0,-14.515717,11.020691,10.0,0.0,1.020691,8.917603,21.6,...,18393.994679,18393.994679,18393.994679,18393.994679,18393.994679,18393.994679,18393.994679,18393.994679,18393.994679,3828.458551
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-06-30 21:56:00+00:00,-156.075592,154.0,0.0,-310.075592,206.665222,204.4,0.0,2.265222,161.529112,347.4,...,63419.604842,63419.604842,14694.700012,10094.235144,36964.579851,63419.604842,63419.604842,63419.604842,63419.604842,7403.527380
2023-06-30 21:57:00+00:00,-156.075592,154.0,0.0,-310.075592,207.082977,204.4,0.0,2.682977,161.229177,347.4,...,71484.204809,71484.204809,13525.193847,9469.995091,42071.344930,71484.204809,71484.204809,71484.204809,71484.204809,8180.851681
2023-06-30 21:58:00+00:00,-156.075592,154.0,0.0,-310.075592,206.958008,204.4,0.0,2.558008,161.479126,347.4,...,79400.969283,79400.969283,12060.877767,9644.652228,41135.723942,79400.969283,79400.969283,79400.969283,79400.969283,8951.180265


In [26]:
steel = assets.loc[:,[x for x in assets.columns if "steel" in x.lower()]]
del(assets)
steel

,Steel Site 1 Production,Steel Site 1 Schedule,Steel Site 1 Deviation,Steel Site 2 Production,Steel Site 2 Schedule,Steel Site 2 Deviation,Steel Site 3 Production,Steel Site 3 Schedule,Steel Site 3 Deviation,Steel Site 4 Production,...,Steel Site 7 Production,Steel Site 7 Schedule,Steel Site 7 Deviation,Steel Site 1 Deviation reactivity,Steel Site 2 Deviation reactivity,Steel Site 3 Deviation reactivity,Steel Site 4 Deviation reactivity,Steel Site 5 Deviation reactivity,Steel Site 6 Deviation reactivity,Steel Site 7 Deviation reactivity
Step date time,,,,,,,,,,,,,,,,,,,,,
2022-12-01 21:44:00+00:00,-0.598816,0.0,-0.598816,1.190521,-5.6,6.790521,2.411269,-3.1,5.511269,7.325202,...,282.496872,-185.8,468.296872,310428.515857,310428.515857,172798.129164,28519.641860,31114.437700,17651.487841,11749.622996
2022-12-01 21:45:00+00:00,-0.598816,0.0,-0.598816,1.190521,-5.6,6.790521,2.320423,-3.1,5.420423,6.984205,...,293.513950,-185.8,479.313950,257858.918055,257858.918055,140693.597808,23210.493540,26337.621114,16565.487677,11884.485459
2022-12-01 21:46:00+00:00,-0.598816,0.0,-0.598816,1.190521,-5.6,6.790521,2.216682,-3.1,5.316682,11.499841,...,300.372788,-185.8,486.172788,197225.888436,197225.888436,108546.800468,17169.514864,20833.594209,15610.893155,11929.874115
2022-12-01 21:47:00+00:00,-0.598816,0.0,-0.598816,1.190521,-5.6,6.790521,2.358518,-3.1,5.458518,7.748986,...,303.845099,-185.8,489.645099,134440.772917,134440.772917,73617.274209,9648.220464,17417.111418,14321.696766,12261.115570
2022-12-01 21:48:00+00:00,-0.598816,0.0,-0.598816,1.190521,-5.6,6.790521,2.619338,-3.1,5.719338,7.018310,...,297.286328,-185.8,483.086328,18393.994679,18393.994679,10074.377214,1225.313813,3199.479417,4155.692975,53567.465861
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-06-30 21:56:00+00:00,-0.898788,-38.7,37.801212,-4.396942,-13.1,8.703058,61.008164,-33.5,94.508164,8.864507,...,24.731481,-112.9,137.631481,1934.591450,63419.604842,3173.753115,7429.536489,63419.604842,11164.542250,13635.672902
2023-06-30 21:57:00+00:00,-0.898788,-38.7,37.801212,-4.396942,-13.1,8.703058,60.898709,-33.5,94.398709,7.559023,...,22.545212,-112.9,135.445212,2092.942209,71484.204809,3401.083526,8575.493609,71484.204809,12287.477205,15800.916281
2023-06-30 21:58:00+00:00,-0.898788,-38.7,37.801212,-4.396942,-13.1,8.703058,63.525810,-33.5,97.025810,6.755251,...,23.902721,-112.9,136.802721,2248.450319,79400.969283,3612.150826,9829.085523,79400.969283,17336.250843,14356.545603


In [27]:
steel["total_steel_production"] = (steel
        .loc[:,[x for x in steel.columns if "production" in x.lower()]]
        .sum(axis = 1)
        )
steel["total_steel_schedule"] = (steel
        .loc[:,[x for x in steel.columns if "schedule" in x.lower()]]
        .sum(axis = 1)
        )
steel["total_steel_deviation"] = (steel
        .loc[:,[x for x in steel.columns if ("deviation" in x.lower()) and (not "reactivity" in x.lower())]]
        .sum(axis = 1)
        )
steel

,Steel Site 1 Production,Steel Site 1 Schedule,Steel Site 1 Deviation,Steel Site 2 Production,Steel Site 2 Schedule,Steel Site 2 Deviation,Steel Site 3 Production,Steel Site 3 Schedule,Steel Site 3 Deviation,Steel Site 4 Production,...,Steel Site 1 Deviation reactivity,Steel Site 2 Deviation reactivity,Steel Site 3 Deviation reactivity,Steel Site 4 Deviation reactivity,Steel Site 5 Deviation reactivity,Steel Site 6 Deviation reactivity,Steel Site 7 Deviation reactivity,total_steel_production,total_steel_schedule,total_steel_deviation
Step date time,,,,,,,,,,,,,,,,,,,,,
2022-12-01 21:44:00+00:00,-0.598816,0.0,-0.598816,1.190521,-5.6,6.790521,2.411269,-3.1,5.511269,7.325202,...,310428.515857,310428.515857,172798.129164,28519.641860,31114.437700,17651.487841,11749.622996,372.830952,-269.1,641.930952
2022-12-01 21:45:00+00:00,-0.598816,0.0,-0.598816,1.190521,-5.6,6.790521,2.320423,-3.1,5.420423,6.984205,...,257858.918055,257858.918055,140693.597808,23210.493540,26337.621114,16565.487677,11884.485459,386.288854,-269.1,655.388854
2022-12-01 21:46:00+00:00,-0.598816,0.0,-0.598816,1.190521,-5.6,6.790521,2.216682,-3.1,5.316682,11.499841,...,197225.888436,197225.888436,108546.800468,17169.514864,20833.594209,15610.893155,11929.874115,396.009448,-269.1,665.109448
2022-12-01 21:47:00+00:00,-0.598816,0.0,-0.598816,1.190521,-5.6,6.790521,2.358518,-3.1,5.458518,7.748986,...,134440.772917,134440.772917,73617.274209,9648.220464,17417.111418,14321.696766,12261.115570,390.990939,-269.1,660.090939
2022-12-01 21:48:00+00:00,-0.598816,0.0,-0.598816,1.190521,-5.6,6.790521,2.619338,-3.1,5.719338,7.018310,...,18393.994679,18393.994679,10074.377214,1225.313813,3199.479417,4155.692975,53567.465861,384.818605,-269.1,653.918605
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-06-30 21:56:00+00:00,-0.898788,-38.7,37.801212,-4.396942,-13.1,8.703058,61.008164,-33.5,94.508164,8.864507,...,1934.591450,63419.604842,3173.753115,7429.536489,63419.604842,11164.542250,13635.672902,96.191218,-240.4,336.591218
2023-06-30 21:57:00+00:00,-0.898788,-38.7,37.801212,-4.396942,-13.1,8.703058,60.898709,-33.5,94.398709,7.559023,...,2092.942209,71484.204809,3401.083526,8575.493609,71484.204809,12287.477205,15800.916281,93.822374,-240.4,334.222374
2023-06-30 21:58:00+00:00,-0.898788,-38.7,37.801212,-4.396942,-13.1,8.703058,63.525810,-33.5,97.025810,6.755251,...,2248.450319,79400.969283,3612.150826,9829.085523,79400.969283,17336.250843,14356.545603,90.306160,-240.4,330.706160


In [28]:
results_list = []
y = residuals
for col, value in steel.items():
    for lag in range(61):
        X = value.shift(
            freq = timedelta(minutes = lag)
        )
        y_X = pd.merge(
            y,
            X,
            left_index = True,
            right_index = True,
            how = "inner",
        )
        results_list.append(
            {
                "variable": col,
                "lag": lag,
                "mi": mutual_info_regression(
                    X = y_X.loc[:,[X.name]],
                    y = y_X.loc[:,y.squeeze().name],
                )[0]
            },            
        )
        print(f"{col} lag {lag} done.")
    

Steel Site 1 Production lag 0 done.
Steel Site 1 Production lag 1 done.
Steel Site 1 Production lag 2 done.
Steel Site 1 Production lag 3 done.
Steel Site 1 Production lag 4 done.
Steel Site 1 Production lag 5 done.
Steel Site 1 Production lag 6 done.
Steel Site 1 Production lag 7 done.
Steel Site 1 Production lag 8 done.
Steel Site 1 Production lag 9 done.
Steel Site 1 Production lag 10 done.
Steel Site 1 Production lag 11 done.
Steel Site 1 Production lag 12 done.
Steel Site 1 Production lag 13 done.
Steel Site 1 Production lag 14 done.
Steel Site 1 Production lag 15 done.
Steel Site 1 Production lag 16 done.
Steel Site 1 Production lag 17 done.
Steel Site 1 Production lag 18 done.
Steel Site 1 Production lag 19 done.
Steel Site 1 Production lag 20 done.
Steel Site 1 Production lag 21 done.
Steel Site 1 Production lag 22 done.
Steel Site 1 Production lag 23 done.
Steel Site 1 Production lag 24 done.
Steel Site 1 Production lag 25 done.
Steel Site 1 Production lag 26 done.
Steel Site 

In [29]:
results = pd.DataFrame(results_list).sort_values(by = "mi", ascending = False)
results

,variable,lag,mi
1245,Steel Site 7 Deviation,25,0.024457
317,Steel Site 2 Deviation,12,0.024250
1102,Steel Site 7 Production,4,0.021263
1708,total_steel_production,0,0.020923
606,Steel Site 4 Production,57,0.020715
...,...,...,...
415,Steel Site 3 Production,49,0.000000
381,Steel Site 3 Production,15,0.000000
330,Steel Site 2 Deviation,25,0.000000
335,Steel Site 2 Deviation,30,0.000000


In [34]:
X_plot = steel[["Steel Site 7 Deviation"]].shift(freq = timedelta(minutes = 25))
X_plot

,Steel Site 7 Deviation
Step date time,
2022-12-01 21:44:00+00:00,468.296872
2022-12-01 21:45:00+00:00,479.313950
2022-12-01 21:46:00+00:00,486.172788
2022-12-01 21:47:00+00:00,489.645099
2022-12-01 21:48:00+00:00,483.086328
...,...
2023-06-30 21:56:00+00:00,137.631481
2023-06-30 21:57:00+00:00,135.445212
2023-06-30 21:58:00+00:00,136.802721
